In [5]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pandas as pd

In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [7]:
cap = cv2.VideoCapture(0)
ls = []

with mp_hands.Hands(
    model_complexity = 0,
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5
) as hands:
    
    while cap.isOpened():
        lmk_x, lmk_y, lmk_z = 0, 0, 0
        wlmk_x, wlmk_y, wlmk_z = 0, 0, 0

        time_start = time.time()

        retval, frame = cap.read()

        if not retval:
            print("Camera Error; Exiting")
            break

        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:

                lmk_x, lmk_y, lmk_z = [[lmk.x, lmk.y, lmk.z] for lmk in hand_landmarks.landmark][0]

                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

        if results.multi_hand_world_landmarks:
            for hand_landmarks in results.multi_hand_world_landmarks:
                wlmk_x, wlmk_y, wlmk_z = [[lmk.x, lmk.y, lmk.z] for lmk in hand_landmarks.landmark][0]

                ls.append([[lmk.x, lmk.y] for lmk in hand_landmarks.landmark])


        frame = cv2.copyMakeBorder(frame, 0, 80, 0, 0, cv2.BORDER_CONSTANT)

        fps = 1 / (time.time() - time_start)

        frame = cv2.putText(frame, f"FPS: {round(fps)}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
        frame = cv2.putText(frame, f"L: {lmk_x:.4f}, {lmk_y:.4f}, {lmk_z:.4f}", (10,frame.shape[1]-90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
        frame = cv2.putText(frame, f"WL: {wlmk_x:.4f}, {wlmk_y:.4f}, {wlmk_z:.4f}", (10,frame.shape[1]-130), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)

        cv2.imshow("Hand Landmarks", frame)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [8]:
cap.release()
cv2.destroyAllWindows()

In [9]:
ls = np.array(ls)
ls = ls.reshape(-1, 42)
ls[:5]

array([[ 0.00772233,  0.05999971,  0.02454698,  0.03208907,  0.03864972,
         0.01176152,  0.04836433, -0.00705937,  0.05084202, -0.02124037,
         0.01635749, -0.01222288,  0.01467768, -0.03438256,  0.01513621,
        -0.05426402,  0.01637551, -0.07467087,  0.00234061, -0.00325448,
        -0.00113547, -0.03003502, -0.00151018, -0.05382849, -0.00557284,
        -0.07519286, -0.01080897,  0.00696477, -0.01770475, -0.01380248,
        -0.02173349, -0.03673345, -0.02356342, -0.05413804, -0.02203409,
         0.02285602, -0.03350209,  0.00668956, -0.03987806, -0.01081933,
        -0.03980641, -0.02102102],
       [ 0.00612337,  0.06467301,  0.02599587,  0.03669615,  0.04021929,
         0.01832211,  0.05451439, -0.00329753,  0.06222405, -0.01819393,
         0.01622401, -0.01247538,  0.01826414, -0.03846272,  0.02234804,
        -0.05927929,  0.02842454, -0.07561393,  0.0020321 , -0.0037099 ,
        -0.0012313 , -0.03338096, -0.00010826, -0.06045076, -0.00232293,
        -0.08238

In [10]:
df_columns = []
for i in range(21):
    df_columns.append(f"Landmark {i} x")
    df_columns.append(f"Landmark {i} y")

In [12]:
# create dataframe where rows are frames and columns are coordinates of all 21 landmarks (x, y have distinct columns)
df = pd.DataFrame(ls, columns=df_columns)
df.shape

(89, 42)

In [13]:
df.to_csv("test_lattices.csv", index=False)